# Retail360 Intelligence — Exploratory Data Analysis

**Dataset:** Brazilian E-Commerce (Olist) · 2016–2018  
**Scope:** 96K orders · 93K customers · 27 states · 74 product categories

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

sns.set_theme(style='whitegrid', palette='Blues_d')
plt.rcParams.update({'figure.dpi': 120, 'font.family': 'DejaVu Sans'})

DB_URL = os.getenv('RETAIL360_DB_URL')
engine = create_engine(DB_URL)

def q(sql):
    return pd.read_sql(sql, engine)

## 1. Data Overview

In [ ]:
overview = q("""
    SELECT
        COUNT(DISTINCT order_id)                        AS total_orders,
        COUNT(DISTINCT customer_key)                    AS total_customers,
        ROUND(SUM(total_order_value)::numeric, 2)       AS gross_revenue,
        ROUND(AVG(total_order_value)::numeric, 2)       AS avg_order_value,
        ROUND(AVG(review_score)::numeric, 2)            AS avg_review_score,
        ROUND(AVG(delivery_days_actual)::numeric, 1)    AS avg_delivery_days,
        SUM(CASE WHEN is_late_delivery THEN 1 ELSE 0 END) AS late_deliveries
    FROM fact_orders
    WHERE order_status_raw = 'delivered'
""")
overview.T.rename(columns={0: 'Value'})

In [ ]:
yearly = q("""
    SELECT
        dd.year,
        COUNT(DISTINCT fo.order_id)                     AS orders,
        COUNT(DISTINCT dc.customer_unique_id)           AS customers,
        ROUND(SUM(fo.total_order_value)::numeric, 0)    AS revenue
    FROM fact_orders fo
    JOIN dim_date dd ON fo.purchase_date_key = dd.date_key
    JOIN dim_customer dc ON fo.customer_key = dc.customer_key
    WHERE fo.order_status_raw = 'delivered'
    GROUP BY dd.year
    ORDER BY dd.year
""")
yearly

## 2. Revenue & Order Trends

In [ ]:
monthly = q("""
    SELECT
        MAKE_DATE(dd.year, dd.month, 1)                 AS month_date,
        dd.year,
        dd.month,
        COUNT(DISTINCT fo.order_id)                     AS orders,
        ROUND(SUM(fo.total_order_value)::numeric, 0)    AS revenue,
        ROUND(AVG(fo.review_score)::numeric, 2)         AS avg_review
    FROM fact_orders fo
    JOIN dim_date dd ON fo.purchase_date_key = dd.date_key
    WHERE fo.order_status_raw = 'delivered'
    GROUP BY dd.year, dd.month
    ORDER BY month_date
""")

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].fill_between(monthly['month_date'], monthly['revenue'], alpha=0.3, color='#1565C0')
axes[0].plot(monthly['month_date'], monthly['revenue'], color='#1565C0', linewidth=2)
axes[0].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'R${x/1e6:.1f}M'))
axes[0].set_title('Monthly Revenue', fontweight='bold')
axes[0].set_ylabel('Revenue')

axes[1].bar(monthly['month_date'], monthly['orders'], color='#1565C0', alpha=0.7, width=20)
axes[1].set_title('Monthly Orders', fontweight='bold')
axes[1].set_ylabel('Orders')
axes[1].set_xlabel('Month')

plt.tight_layout()
plt.show()

In [ ]:
quarterly = q("""
    SELECT
        dd.year,
        dd.quarter,
        ROUND(SUM(fo.total_order_value)::numeric, 0) AS revenue,
        COUNT(DISTINCT fo.order_id) AS orders
    FROM fact_orders fo
    JOIN dim_date dd ON fo.purchase_date_key = dd.date_key
    WHERE fo.order_status_raw = 'delivered'
    GROUP BY dd.year, dd.quarter
    ORDER BY dd.year, dd.quarter
""")

pivot = quarterly.pivot(index='quarter', columns='year', values='revenue')

ax = pivot.plot(kind='bar', figsize=(10, 5), colormap='Blues', edgecolor='white')
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'R${x/1e6:.1f}M'))
ax.set_title('Revenue by Quarter & Year', fontweight='bold')
ax.set_xlabel('Quarter')
ax.set_ylabel('Revenue')
ax.legend(title='Year')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## 3. Customer Segmentation (RFM)

In [ ]:
segments = q("""
    SELECT
        segment_group,
        customer_segment,
        COUNT(*) AS customers,
        ROUND(AVG(monetary)::numeric, 2) AS avg_monetary,
        ROUND(AVG(recency_days)::numeric, 0) AS avg_recency_days,
        ROUND(AVG(frequency)::numeric, 1) AS avg_frequency
    FROM vw_customer_segments
    GROUP BY segment_group, customer_segment
    ORDER BY avg_monetary DESC
""")
segments

In [ ]:
group_summary = q("""
    SELECT
        segment_group,
        COUNT(*) AS customers,
        ROUND(SUM(monetary)::numeric, 0) AS total_revenue,
        ROUND(AVG(monetary)::numeric, 2) AS avg_clv
    FROM vw_customer_segments
    GROUP BY segment_group
    ORDER BY total_revenue DESC
""")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = ['#1565C0', '#42A5F5', '#E65100', '#BDBDBD']

axes[0].pie(
    group_summary['customers'],
    labels=group_summary['segment_group'],
    autopct='%1.1f%%',
    colors=colors,
    startangle=90,
    wedgeprops={'edgecolor': 'white', 'linewidth': 2}
)
axes[0].set_title('Customer Distribution by Segment', fontweight='bold')

axes[1].barh(group_summary['segment_group'], group_summary['total_revenue'], color=colors)
axes[1].xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'R${x/1e6:.1f}M'))
axes[1].set_title('Total Revenue by Segment', fontweight='bold')
axes[1].set_xlabel('Revenue')

plt.tight_layout()
plt.show()

In [ ]:
rfm_scatter = q("""
    SELECT recency_days, frequency, monetary, segment_group
    FROM vw_customer_segments
    WHERE monetary < 5000
""")

palette = {
    'High Value': '#1565C0',
    'Growth': '#42A5F5',
    'At Risk': '#C62828',
    'Low Priority': '#BDBDBD'
}

fig, ax = plt.subplots(figsize=(12, 6))
for group, df_g in rfm_scatter.groupby('segment_group'):
    ax.scatter(
        df_g['recency_days'], df_g['monetary'],
        s=df_g['frequency'] * 15,
        alpha=0.5,
        label=group,
        color=palette.get(group, '#999')
    )

ax.set_xlabel('Recency (days since last order)')
ax.set_ylabel('Monetary Value (R$)')
ax.set_title('RFM Scatter — Recency vs Monetary (bubble = Frequency)', fontweight='bold')
ax.legend(title='Segment')
plt.tight_layout()
plt.show()

## 4. Logistics & Delivery Performance

In [ ]:
geo = q("""
    SELECT
        state,
        total_orders,
        total_revenue,
        avg_delivery_days,
        late_delivery_pct,
        avg_review_score
    FROM vw_geo_performance
    ORDER BY total_revenue DESC
    LIMIT 15
""")

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

colors_late = ['#C62828' if x > 10 else '#E65100' if x > 5 else '#00897B' for x in geo['late_delivery_pct']]
axes[0].barh(geo['state'], geo['late_delivery_pct'], color=colors_late)
axes[0].axvline(x=10, color='#C62828', linestyle='--', linewidth=1, label='10% threshold')
axes[0].set_title('Late Delivery % by State (Top 15 by Revenue)', fontweight='bold')
axes[0].set_xlabel('Late Delivery %')
axes[0].legend()

axes[1].barh(geo['state'], geo['avg_delivery_days'], color='#1565C0', alpha=0.7)
axes[1].set_title('Avg Delivery Days by State', fontweight='bold')
axes[1].set_xlabel('Days')

plt.tight_layout()
plt.show()

In [ ]:
delivery_review = q("""
    SELECT
        CASE
            WHEN delivery_days_actual <= 5  THEN '0-5 days'
            WHEN delivery_days_actual <= 10 THEN '6-10 days'
            WHEN delivery_days_actual <= 20 THEN '11-20 days'
            ELSE '20+ days'
        END AS delivery_bucket,
        ROUND(AVG(review_score)::numeric, 2) AS avg_review,
        COUNT(*) AS orders
    FROM fact_orders
    WHERE order_status_raw = 'delivered' AND delivery_days_actual IS NOT NULL
    GROUP BY delivery_bucket
    ORDER BY MIN(delivery_days_actual)
""")

fig, ax1 = plt.subplots(figsize=(10, 5))
ax2 = ax1.twinx()

ax1.bar(delivery_review['delivery_bucket'], delivery_review['orders'], color='#1565C0', alpha=0.6, label='Orders')
ax2.plot(delivery_review['delivery_bucket'], delivery_review['avg_review'], color='#C62828', marker='o', linewidth=2, label='Avg Review')

ax1.set_ylabel('Order Count')
ax2.set_ylabel('Avg Review Score')
ax2.set_ylim(1, 5)
ax1.set_title('Delivery Speed vs Customer Satisfaction', fontweight='bold')

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2)

plt.tight_layout()
plt.show()

## 5. Product Category Analysis

In [ ]:
categories = q("""
    SELECT
        dp.category_name_en AS category,
        COUNT(DISTINCT fo.order_id) AS orders,
        ROUND(SUM(fo.price)::numeric, 0) AS revenue,
        ROUND(AVG(fo.price)::numeric, 2) AS avg_price,
        ROUND(AVG(fo.review_score)::numeric, 2) AS avg_review
    FROM fact_orders fo
    JOIN dim_product dp ON fo.product_key = dp.product_key
    WHERE fo.order_status_raw = 'delivered'
      AND dp.category_name_en IS NOT NULL
    GROUP BY dp.category_name_en
    ORDER BY revenue DESC
    LIMIT 20
""")

fig, ax = plt.subplots(figsize=(12, 8))
ax.barh(categories['category'], categories['revenue'], color='#1565C0', alpha=0.8)
ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'R${x/1e6:.1f}M'))
ax.set_title('Top 20 Product Categories by Revenue', fontweight='bold')
ax.set_xlabel('Revenue')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
pareto = q("""
    SELECT
        dp.category_name_en AS category,
        ROUND(SUM(fo.price)::numeric, 0) AS revenue
    FROM fact_orders fo
    JOIN dim_product dp ON fo.product_key = dp.product_key
    WHERE fo.order_status_raw = 'delivered'
      AND dp.category_name_en IS NOT NULL
    GROUP BY dp.category_name_en
    ORDER BY revenue DESC
""")

pareto['cumulative_pct'] = pareto['revenue'].cumsum() / pareto['revenue'].sum() * 100
pareto['rank_pct'] = (pareto.index + 1) / len(pareto) * 100

fig, ax1 = plt.subplots(figsize=(12, 5))
ax2 = ax1.twinx()

ax1.bar(range(len(pareto)), pareto['revenue'], color='#1565C0', alpha=0.6)
ax2.plot(range(len(pareto)), pareto['cumulative_pct'], color='#C62828', linewidth=2)
ax2.axhline(y=80, color='#E65100', linestyle='--', linewidth=1, label='80% revenue')

ax1.set_xlabel('Category Rank')
ax1.set_ylabel('Revenue (R$)')
ax2.set_ylabel('Cumulative Revenue %')
ax2.set_ylim(0, 105)
ax1.set_title('Pareto Analysis — Product Categories', fontweight='bold')
ax2.legend()

plt.tight_layout()
plt.show()

cutoff = pareto[pareto['cumulative_pct'] <= 80]
print(f"{len(cutoff)} categories ({len(cutoff)/len(pareto)*100:.1f}%) generate 80% of revenue")

## 6. Payment Behavior Analysis

In [ ]:
payments = q("""
    SELECT
        dpt.payment_type,
        COUNT(DISTINCT fp.order_id)                     AS transactions,
        ROUND(SUM(fp.payment_value)::numeric, 0)        AS total_value,
        ROUND(AVG(fp.payment_value)::numeric, 2)        AS avg_value,
        ROUND(AVG(fp.payment_installments)::numeric, 1) AS avg_installments
    FROM fact_payments fp
    JOIN dim_payment_type dpt ON fp.payment_type_key = dpt.payment_type_key
    GROUP BY dpt.payment_type
    ORDER BY total_value DESC
""")
payments

In [ ]:
installments = q("""
    SELECT
        payment_installments,
        COUNT(*) AS transactions,
        ROUND(AVG(payment_value)::numeric, 2) AS avg_value
    FROM fact_payments fp
    JOIN dim_payment_type dpt ON fp.payment_type_key = dpt.payment_type_key
    WHERE dpt.payment_type = 'credit_card'
      AND payment_installments BETWEEN 1 AND 12
    GROUP BY payment_installments
    ORDER BY payment_installments
""")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors_pay = ['#1565C0', '#42A5F5', '#00897B', '#E65100']
axes[0].pie(
    payments['total_value'],
    labels=payments['payment_type'],
    autopct='%1.1f%%',
    colors=colors_pay,
    startangle=90,
    wedgeprops={'edgecolor': 'white', 'linewidth': 2}
)
axes[0].set_title('Payment Method Share by Value', fontweight='bold')

axes[1].bar(installments['payment_installments'], installments['transactions'], color='#1565C0', alpha=0.8)
axes[1].set_title('Credit Card Installment Distribution', fontweight='bold')
axes[1].set_xlabel('Number of Installments')
axes[1].set_ylabel('Transactions')
axes[1].set_xticks(range(1, 13))

plt.tight_layout()
plt.show()

## 7. Key Findings Summary

In [ ]:
kpis = q("""
    SELECT
        ROUND(SUM(total_order_value)::numeric / 1e6, 2)     AS revenue_m,
        COUNT(DISTINCT order_id)                             AS total_orders,
        ROUND(AVG(review_score)::numeric, 2)                AS avg_review,
        ROUND(
            (1 - SUM(CASE WHEN is_late_delivery THEN 1 ELSE 0 END)::numeric
            / NULLIF(COUNT(CASE WHEN delivered_timestamp IS NOT NULL THEN 1 END), 0)) * 100, 2
        ) AS otd_rate
    FROM fact_orders
    WHERE order_status_raw = 'delivered'
""")

print(f"Total Revenue    : R$ {kpis['revenue_m'].iloc[0]}M")
print(f"Total Orders     : {kpis['total_orders'].iloc[0]:,}")
print(f"Avg Review Score : {kpis['avg_review'].iloc[0]} / 5.0")
print(f"OTD Rate         : {kpis['otd_rate'].iloc[0]}%")